In [ ]:
!pip install yfinance
!pip install pykrx

# **yahoo finance**
---
> Oversea

In [ ]:
import yfinance as yf
import pandas as pd
import time

### Main

In [ ]:
df = yf.download("TSLL", "2019-01-02", "2024-12-31")
df

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['TSLL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,TSLL,TSLL,TSLL,TSLL,TSLL,TSLL
Date,,,,,,


In [ ]:
def YF(url: str,
          sheet: str = 'sheet1',
          _from: time.strftime("%YY-%mm-%dd") = "2019-01-01",
          _to: time.strftime("%YY-%mm-%dd") = "2024-12-31",
          dir: str = "uncategorized"
          ):
  '''
    Parameters
    ----------
    `url`: df/labels 폴더 안 스프레드시트 파일 url

    `sheet`: 스프레드시트 안 시트 이름, 기본 "sheet1", 그냥 그대로 두면

    `_from`: 긁어오는 시작 기간, 기본 "2019-01-01"

    `_to`: 긁어오는 마지막 기간, 기본 "2024-12-31"

    `dir`: 저장 폴더명 (df 폴더 안에 폴더 이름만), 기본 "uncategorized"

    Examples
    --------
    >>> !pip install import_ipynb
    >>> import import_ipynb
    >>> import Data_Downloader
    >>> Data_Donloader.YF("https://docs.google.com/spreadsheets/d/1roVj96tWjSR7KXEsJaD2H1YBf0RRF-hWCr6KPZK54J8/edit?gid=10678020#gid=10678020", dir="oversea")
  '''
  url = url[:url.find("/edit?gid=")]
  sheet_name = sheet
  url = f"{url}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
  codes = pd.read_csv(url)

  for i in range(len(codes)):
    df = yf.download(codes["Symbol"].iloc[i], _from, _to)
    df = df[['Low', 'Open', 'Close', 'High', 'Volume']]
    df.columns.names = [None, None]
    df = df.droplevel(level=1, axis=1)
    df.to_csv("./df/{0}/{1}.csv".format(dir, codes["Symbol"].iloc[i]), encoding='utf-8-sig')
    print(i, codes['Symbol'].iloc[i])


# **Naver / KRX**
---
> Domestic

In [ ]:
import pandas as pd
from pykrx import stock, bond
import time
from datetime import datetime

### Main

In [ ]:
def code_correct(code):
  code = str(code)
  if len(code) < 6:
    return "0"*(6-len(code)) + code

  elif len(code) == 6:
    return code



# def timestamp_index(year: int = 2019, n: int = 6):
#   year = year
#   li = []
#   for i in range(n):
#     for month in range(1, 13):
#       li.append(pd.Timestamp(str(year)+"-"+str(month)+'-01'))
#     year += 1
#   return li



def PYKRX(url: str,
          sheet: str = 'sheet1',
          _from: time.strftime("%YY-%mm-%dd") = "2019-01-01",
          _to: time.strftime("%YY-%mm-%dd") = "2024-12-31",
          dir: str = "uncategorized"
          ):
  '''
    Parameters
    ----------
    `url`: df/labels 폴더 안 스프레드시트 파일 url

    `sheet`: 스프레드시트 안 시트 이름, 기본 "sheet1", 그냥 그대로 두면

    `_from`: 긁어오는 시작 기간, 기본 "2019-01-01"

    `_to`: 긁어오는 마지막 기간, 기본 "2024-12-31"

    `dir`: 저장 폴더명 (df 폴더 안에 폴더 이름만), 기본 "uncategorized"

    Examples
    --------
    >>> !pip install import_ipynb
    >>> import import_ipynb
    >>> import Data_Downloader
    >>> Data_Donloader.PYKRX("https://docs.google.com/spreadsheets/d/1roVj96tWjSR7KXEsJaD2H1YBf0RRF-hWCr6KPZK54J8/edit?gid=10678020#gid=10678020", dir="domestic")

    - ETF, 채권은 각각 K_ETF(), K_BOND()로
  '''
  url = url[:url.find("/edit?gid=")]
  sheet_name = sheet
  url = f"{url}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
  codes = pd.read_csv(url)

  for i in range(len(codes)):
    time.sleep(0.1)   # 빨리해도되는데 민폐 / 시간들여서 천천히
    df = stock.get_market_ohlcv(_from, _to, code_correct(codes["Symbol"].iloc[i]))
    df.drop(['등락률'], axis=1)
    df = df[['저가', '시가', '종가', '고가', '거래량']]
    df.columns = ['Low', 'Open', 'Close', 'High', 'Volume']
    df.index.name="Date"
    # df.index = timestamp_index()
    name = codes["Name"].iloc[i]
    df.to_csv(f"./df/{dir}/{name}.csv", encoding='utf-8-sig') # instance needed for f''
    print(i, name)



def K_ETF(url: str,
          sheet: str = 'sheet1',
          _from: time.strftime("%YY-%mm-%dd") = "2019-01-01",
          _to: time.strftime("%YY-%mm-%dd") = "2024-12-31",
          dir: str = "uncategorized"
          ):
  '''
    Parameters
    ----------
    `url`: df/labels 폴더 안 스프레드시트 파일 url

    `sheet`: 스프레드시트 안 시트 이름, 기본 "sheet1", 그냥 그대로 두면

    `_from`: 긁어오는 시작 기간, 기본 "2019-01-01"

    `_to`: 긁어오는 마지막 기간, 기본 "2024-12-31"

    `dir`: 저장 폴더명 (df 폴더 안에 폴더 이름만), 기본 "uncategorized"

    Examples
    --------
    >>> !pip install import_ipynb
    >>> import import_ipynb
    >>> import Data_Downloader
    >>> Data_Donloader.K_ETF("https://docs.google.com/spreadsheets/d/1roVj96tWjSR7KXEsJaD2H1YBf0RRF-hWCr6KPZK54J8/edit?gid=10678020#gid=10678020", dir="k_etf")

    - ETF, 채권은 각각 K_ETF(), K_BOND()로

    - 참고: https://github.com/sharebook-kr/pykrx?tab=readme-ov-file#22-etx-api
  '''
  url = url[:url.find("/edit?gid=")]
  sheet_name = sheet
  url = f"{url}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
  codes = pd.read_csv(url)

  for i in range(len(codes)):
    time.sleep(0.1)   # 빨리해도되는데 민폐 / 시간들여서 천천히
    df = stock.get_etf_ohlcv_by_date(_from, _to, code_correct(codes["Symbol"].iloc[i]))
    df.drop(['거래대금', '기초지수'], axis=1)
    df = df[["NAV", '저가', '시가', '종가', '고가', '거래량']]
    df.columns = ["NAV", 'Low', 'Open', 'Close', 'High', 'Volume']
    df.index.name="Date"
    # df.index = timestamp_index()
    name = codes["Name"].iloc[i]
    df.to_csv(f"./df/{dir}/{name}.csv", encoding='utf-8-sig') # instance needed for f''
    print(i, name)



def K_BOND(url: str,
          sheet: str = 'sheet1',
          _from: time.strftime("%YY-%mm-%dd") = "2019-01-01",
          _to: time.strftime("%YY-%mm-%dd") = "2024-12-31",
          dir: str = "uncategorized"
          ):
  '''
    Parameters
    ----------
    `url`: df/labels 폴더 안 스프레드시트 파일 url

    `sheet`: 스프레드시트 안 시트 이름, 기본 "sheet1", 그냥 그대로 두면

    `_from`: 긁어오는 시작 기간, 기본 "2019-01-01"

    `_to`: 긁어오는 마지막 기간, 기본 "2024-12-31"

    `dir`: 저장 폴더명 (df 폴더 안에 폴더 이름만), 기본 "uncategorized"

    Examples
    --------
    >>> !pip install import_ipynb
    >>> import import_ipynb
    >>> import Data_Downloader
    >>> Data_Donloader.K_BOND("https://docs.google.com/spreadsheets/d/1roVj96tWjSR7KXEsJaD2H1YBf0RRF-hWCr6KPZK54J8/edit?gid=10678020#gid=10678020", dir="k_bond")

    - ETF, 채권은 각각 K_ETF(), K_BOND()로

    - 참고: https://github.com/sharebook-kr/pykrx?tab=readme-ov-file#23-%EC%B1%84%EA%B6%8C-api
  '''
  url = url[:url.find("/edit?gid=")]
  sheet_name = sheet
  url = f"{url}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
  codes = pd.read_csv(url)

  for i in range(len(codes)):
    time.sleep(0.1)   # 빨리해도되는데 민폐 / 시간들여서 천천히
    df = bond.get_otc_treasury_yields(_from, _to, codes["Symbol"].iloc[i])
    df = df['수익률']
    df.columns = ['Rate']
    df.index.name="Date"
    name = codes["Name"].iloc[i]
    df.to_csv(f"./df/{dir}/{name}.csv", encoding='utf-8-sig') # instance needed for f''
    print(i, name)